In [2]:
import numpy as np

In [ ]:
# rents out car and is credited with $10 by company
# if he is out of cars then the business is lost
# cars can be rented the day after returned 
# jack can move cars between locations for 2 dollars

# Iterative Policy Evaluation Algorithm
class PolicyIteration(): 
    def __init__(self, init_state_values, starting_actions, starting_state, terminal_state):
        #Initialize state values randomly
        self.evaluated_state_values = init_state_values

        #discount factor
        self.discount = 0.9
        self.move_action_cost = -2.0 

        self.return_lamba = [3, 4]
        self.rent_lamba = [3, 2]

        #Termination States
        self.terminal_state = terminal_state

        #When to stop
        self.termination_diff = 0.00000001
        self.diff = 100

        # select random integer between 0 and 3
        self.current_state = starting_state
        self.current_action = starting_actions

        self.action_probabilitiy = 1
    
    def get_starting_state_random(self):
        x = np.random.randint(0, 20)
        y = np.random.randint(0, 20)
        while ([x, y] == self.terminal_state):
            x = np.random.randint(0, 20)
            y = np.random.randint(0, 20)
        
        self.current_state = [x, y]

    def get_action_cost(self): 
        action = self.current_action[self.current_state[0]][self.current_state[1]]

        return action[1] * self.move_action_cost
    # Get the estimated state values using the action reward and 
    def get_next_state_value(self):
        expected_value = 0.0

        action = self.current_action[self.current_state[0], self.current_state[1]]
        state_after_action = self.get_state_after_action()

        return self.action_probabilitiy * (self.move_action_cost * action[1] + self.discount * self.evaluated_state_values[state_after_action[0], state_after_action[1]])

    def get_state_after_action(self):
        action = self.current_action[self.current_state[0], self.current_state[1]]
        new_state = []
        if action[0] == 0:
            new_state = self.current_state[action[0]] - action[1]
            new_state = self.current_state[action[0]] + action[1]
        else: 
            new_state = self.current_state[action[0]] + action[1]
            new_state = self.current_state[action[0]] - action[1]
        
        return new_state
    
    def take_action(self):
        self.current_state = self.get_state_after_action()

                
    
    def iterative_evaluation(self):
        while (self.diff > self.termination_diff):
            self.diff = 0.0
            while (True):
                if (self.current_state in self.terminal_states):
                    break

                old_state_value = self.evaluated_state_values[self.current_state[0]][self.current_state[1]]
                new_state_value = self.get_next_state_value()

                self.evaluated_state_values[self.current_state[0]][self.current_state[1]] = new_state_value
                self.diff = max(self.diff, abs(old_state_value - new_state_value))

                self.take_action()

            self.current_state = self.get_starting_state_random()

        return self.evaluated_state_values

In [ ]:
#Initialize Problem
terminal_state = [0, 0]
x = np.random.randint(0, 20)
y = np.random.randint(0, 20)
while ([x, y] == terminal_state):
    x = np.random.randint(0, 20)
    y = np.random.randint(0, 20)
starting_state = [x, y]
#Initialize a random number
init_state_values = np.random.rand(20, 20)*100
init_state_values[0][terminal_state] = 0
init_state_values[1][terminal_state] = 0

starting_actions = np.zeros((20, 20, 2, 20))

In [ ]:
# Gives you the expexted value of a state in the grid world
PolicyIteration(init_state_values, starting_actions, starting_state, terminal_state).iterative_evaluation()

In [49]:
init_state_values = np.zeros((20, 20, 2))
# set first index of 3d array to 0 or 1 randomly

In [53]:
#get random 0 or 1
init_state_values[:,:,0] = np.random.randint(0, 2, (20, 20))

In [54]:
init_state_values

array([[[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
      